## Installing needed packages

In [1]:
#conda install -c conda-forge BeautifulSoup4 --yes 
#!conda install -c conda-forge requests --yes 
#!conda install -c conda-forge lxml --yes 
#!conda install -c conda-forge html5lib --yes 
#!conda install -c conda-forge geopy --yes ## already installed


### Importing needed libraries

In [4]:
from bs4 import BeautifulSoup # for webscarping
import requests #to call the link
import numpy as np
import pandas as pd

#for maps
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#import for the lat and lang as the geocoder is not working
from geopy.geocoders import Nominatim

## Part 1 begins here

### Using Beautiful Soup to parse the link after reading it using requests

In [76]:
#!conda install -c conda-forge lxml --yes 
source =  requests.get ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup  = BeautifulSoup(source,'lxml')
#print (soup.prettify())

#### Converting all needed tables from the link in the dataframe

In [60]:
for table in soup.find_all('table',class_='wikitable sortable'):
    s = table
#print(type(s))
st =  str(s)
dfs =  pd.read_html(st)[0]
print(dfs)

    Postcode           Borough           Neighborhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]


## Optional - directly using ipython to display 'table'

In [289]:
#!conda install -c conda-forge ipython --yes 
#from IPython.display import display_html
#display_html(st, raw=True)

### To directly read a table from the link - will be using this for the problem

In [7]:
## using df as the final dataframe for part 1 of the question

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
print(df)

    Postcode           Borough           Neighborhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]


### Renaming the column Postcode to PostalCode to match with the requirement

In [9]:
df = df.rename(columns= {"Postcode": "PostalCode"})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Filtering out the 'Not assigned' Borough values

In [15]:
df_filtered = df[df['Borough']!='Not assigned']
df_filtered

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


#### First copying over the Borough to Neighborhood for Neighborhood values as 'Not assigned'. This will help if we have to aggregate neighborhood values in one row





In [20]:
df_copy = df_filtered.copy()
df_copy.loc[df_copy['Neighborhood'] == 'Not assigned','Neighborhood'] = df_copy['Borough']
#df_copy

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


#### To check if the column value was copied - first check the uncopied dataframe and then the updated i.e. df_copy

In [36]:
print('Rows where Neighborhood and borough are same is:\n ',df_filtered.loc[df_filtered['Neighborhood']== df_filtered['Borough']]) # returns empty as there's no same value
print('Rows where Neighborhood is Not assigned:\n',df_filtered.loc[df_filtered['Neighborhood']== 'Not assigned']) # returns one row where Neighborhood is Not assigned
print('Rows where Neighborhood is copied from Borough:\n',df_copy.loc[df_copy['Neighborhood']== df_copy['Borough']]) # returns the row where neighborhood value was updated

Rows where Neighborhood and borough are same is:
  Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []
Rows where Neighborhood is Not assigned:
   PostalCode       Borough  Neighborhood
7        M7A  Queen's Park  Not assigned
Rows where Neighborhood is copied from Borough:
   PostalCode       Borough  Neighborhood
7        M7A  Queen's Park  Queen's Park


#### Checking the rows where the Neighborhood values would need to be aggregated

In [141]:
df_cnt = df_copy.groupby(['PostalCode','Borough']).count()
print (df_cnt)
print('Final data should have {} rows'.format(df_cnt.shape[0]))

                        Neighborhood
PostalCode Borough                  
M1B        Scarborough             2
M1C        Scarborough             3
M1E        Scarborough             3
M1G        Scarborough             1
M1H        Scarborough             1
...                              ...
M9N        York                    1
M9P        Etobicoke               1
M9R        Etobicoke               4
M9V        Etobicoke               8
M9W        Etobicoke               1

[103 rows x 1 columns]
Final data should have 103 rows


#### Creating the final dataset as per requirements

In [252]:
df_final_part1 =  df_copy.groupby(['PostalCode','Borough']).agg(Neighborhood = ('Neighborhood',', '.join)).reset_index()
print(df_final_part1)

    PostalCode      Borough                                       Neighborhood
0          M1B  Scarborough                                     Rouge, Malvern
1          M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
2          M1E  Scarborough                  Guildwood, Morningside, West Hill
3          M1G  Scarborough                                             Woburn
4          M1H  Scarborough                                          Cedarbrae
..         ...          ...                                                ...
98         M9N         York                                             Weston
99         M9P    Etobicoke                                          Westmount
100        M9R    Etobicoke  Kingsview Village, Martin Grove Gardens, Richv...
101        M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
102        M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


In [253]:
print('Final data has {} rows'.format(df_final_part1.shape[0]))

Final data has 103 rows


## Part 2 begins here

In [167]:
#!conda install -c conda-forge geocoder --yes
import geocoder as gc
print("Geocoder installed and imported.")

Geocoder installed and imported.


##### Trying to ude geocoder but returns "Denied Request error"

In [246]:
 lat_lng = None
while(lat_lng is None) :
        g = gc.google('{}, Toronto, Ontario'.format(df_copy['PostalCode'].to_string()))
        print (g)        #print('{}, Canada'.format(df_copy['PostalCode'].to_string()))
        lat_lng = g.latlng
print(df_copy['Borough'].to_string())

<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>
<[REQUEST_DENIED] Google - Geocode [empty]>


KeyboardInterrupt: 

#### Using the same geo package as in the other exercises but this also fails at some addresses

In [238]:
for i in range(1,df_cnt.shape[0]+1): # for unique count
    #print (i)
    address = df_copy['PostalCode'].iloc[i]+ ', Downtown Toronto, Ontario'
    print(address)
    location = geolocator.geocode(address)
    print(location)
    latitude = location.latitude
    longitude = location.longitude
print('The geograpical coordinate of CA are {}, {}.'.format(latitude, longitude))

M4A, Downtown Toronto, Ontario
None


AttributeError: 'NoneType' object has no attribute 'latitude'

#### Reading from the CSV file provided for the exercise :(

In [271]:
df_csv = pd.read_csv('Geospatial_Coordinates.csv')
df_csv
df_csv = df_csv.rename(columns= {"Postal Code": "PostalCode"})
df_csv.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Above dataframe will need to be joined with df_final to add long and lat values

##### Using merge to add Latitufe 

In [273]:
df_final_part1
df_part2 = pd.merge(df_final_part1, df_csv, left_on ='PostalCode', right_on = 'PostalCode')
df_part2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
